In [ ]:
import re

In [ ]:
def color_keyword(match):
    return f"\033[102m{match.group(0)}\033[00m"

def color_title(match, color=105):
    return f"{match.group(0)}\033[{color}m"

def process_articles(message, keywords):
    pattern = "|".join(keywords)
    dict_kw = {key: [] for key in keywords}
    articles = list(re.finditer(r"\nTitle\: ", message))
    
    for i in range(len(articles)):
        if i == len(articles) - 1:
            article = message[articles[i].start():]
        else:
            article = message[articles[i].start():articles[i+1].start()]
        kw = re.findall(pattern, article, flags=re.IGNORECASE)
        if len(kw)==0:
            continue
        
        article = re.sub(pattern, color_keyword, article, flags=re.IGNORECASE)
        title = article[:list(re.finditer(r"\nAuthors\: ", article))[0].start()]
        title = re.sub(" +", " ", title)
        title = re.sub(r"\n ", r"\n", title)
        title = f"\033[105m{title}\033[00m"
        title = re.sub("(?<=\\033\[102m).*?\\033\[00m", color_title, title, flags=re.IGNORECASE)
        

        url = list(re.finditer("https\:\/\/arxiv\.org\/.*?(?= ,  )", article))[0]
        url = article[url.start():url.end()]

        abstract = list(re.finditer(r"\n\\\\\n  ", article))
        if len(abstract)==0:
            abstract = ""
        else:
            abstract = article[abstract[0].end():list(re.finditer(r"\n\\\\ \( ", article))[0].start()]
            abstract = re.sub(" +", " ", abstract)
            abstract = re.sub(r"\n ", r"\n", abstract)
            
        flag_break = False
        for key in keywords:
            flag_break
            for k in kw:
                if re.search(key, k, flags=re.IGNORECASE) is not None:
                    dict_kw[key] += [{"title": title, "abstract": abstract, "url": url}]
                    flag_break = True
                    break
            if flag_break:
                break
    
    for key in keywords:
        if len(dict_kw[key]) == 0:
            print(f"\033[101m{key}\033[00m: None")
        else:
            print(f"\033[101m{key}\033[00m")
            for article in dict_kw[key]:
                print("\n".join(["\033[100m"+article['title']+"\033[00m", article['abstract'], article['url'], "\n------------------------------------------------------------------------------"]))
    return dict_kw

In [ ]:
keywords = ["Stereo", "Matching", "Multi.?view", "satellite", "Photogrammetry", "3D", "marriage"]

In [ ]:
message = open("msg.txt").read()
dict_kw = process_articles(message, keywords)